In [1]:
# Necessary imports
from datetime import datetime
import pytz
from copy import deepcopy
from matplotlib import pyplot as plt
import matplotlib
import cvxpy as cp
import numpy as np
import pandas as pd
import seaborn as sns
import json
import os
import sys
import random
import heapq  # Priority Queue

from acnportal import acnsim, algorithms
from acnportal.acnsim import analysis
from acnportal.acnsim.events.event import PluginEvent
from acnportal.acnsim.events.event import UnplugEvent
from acnportal.signals.tariffs import TimeOfUseTariff
from adacharge import *
from acnportal.acnsim.interface import Interface, SessionInfo, InfrastructureInfo

In [2]:
# Define common variables for experiment
API_KEY = 'DEMO_TOKEN'
TIMEZONE = pytz.timezone('America/Los_Angeles')
SITE = 'caltech'
PERIOD = 30  # minutes
VOLTAGE = 208  # volts
KW_TO_AMPS = 1000 / 208
KWH_TO_AMP_PERIODS = KW_TO_AMPS * (60 / 5)
MAX_LEN = 144
FORCE_FEASIBLE = True
EVENTS_DIR = "C:\\Users\\s3955218\\repos\\acn-ev-simulation\\events"
VERBOSE = True

### Reading from existing simulation data

In [4]:
event_name = "simevent_1"
event_path = os.path.join(EVENTS_DIR, event_name + '.json')

def event_queue(event_path):
    if os.path.exists(event_path):
        with open(event_path, 'r') as f:
            event = acnsim.EventQueue.from_json(f)
    return event

In [5]:
# width = 1
# if events.get_last_timestamp() is not None:
#     width = events.get_last_timestamp() + 1
# print(width)
# print(events.get_last_timestamp())

# Experimenting with new width
width = 20

### Data model for various component
- EV
- Battery
- EVSE

In [6]:
events = event_queue(event_path)
count = 0
for event in iter(events.queue):
    if count < 5:
        print(event[1].ev)
        print("------------------------------------------")
        print(event[1].ev._battery)
        print("------------------------------------------")
        count = count + 1

acnportal.acnsim.models.ev.EV(_arrival=1, _departure=116, _session_id=<str object at 0x000002677D212810>, _station_id=<str object at 0x000002677D22E6F0>, _requested_energy=37.14, _estimated_departure=116, _battery=<acnportal.acnsim.models.battery.Battery object at 0x000002677D7AD640>, _energy_delivered=0, _current_charging_rate=0)
------------------------------------------
acnportal.acnsim.models.battery.Battery(_capacity=37.14, _current_charge=0, _init_charge=0, _max_power=6.656, _current_charging_power=0)
------------------------------------------
acnportal.acnsim.models.ev.EV(_arrival=2, _departure=146, _session_id=<str object at 0x000002677D212BD0>, _station_id=<str object at 0x000002677D22EA30>, _requested_energy=5.668, _estimated_departure=146, _battery=<acnportal.acnsim.models.battery.Battery object at 0x000002677D7ADDC0>, _energy_delivered=0, _current_charging_rate=0)
------------------------------------------
acnportal.acnsim.models.battery.Battery(_capacity=5.668, _current_ch

### EV grouping logic

### System Model

This is a programmatic model for the system. For this simulation we are using Caltech ACN model

In [7]:
basic_evse=True
caps = 150 # Transformer capacity

network = acnsim.sites.caltech_acn(voltage=VOLTAGE,
                              transformer_cap=caps,
                              basic_evse=basic_evse)

# print(cn.constraint_matrix)
pilot_signals = np.zeros((len(network.station_ids), width))
charging_rates = np.zeros((len(network.station_ids), width))

print(charging_rates)
# print(pilot_signals)
print(charging_rates.shape)
print(pilot_signals.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(54, 20)
(54, 20)


### Event Data 
Real-time data or offline data

In [8]:
copy_events = deepcopy(events)
processed_event = []  # for next stage processing

iteration = 0
while iteration < 20:
    current_event = copy_events.get_event()
    processed_event.append(current_event)
    iteration += 1

### EV Plugin to System (Model)

In [9]:
# event_history = []
ev_history = {}

count = 0
for event in processed_event:
    if event.event_type == "Plugin":
        try:
            print("Plugin event...")
            network.plugin(event.ev)
            ev_history[event.ev.session_id] = event.ev
            copy_events.add_event(UnplugEvent(event.ev.departure, current_event.ev))
            count += 1
        except Exception as e:
            print(e)

print(f"Plugined in EV {count}")

Plugin event...
Plugin event...
Plugin event...
Plugin event...
Plugin event...
Plugin event...
Plugin event...
Plugin event...
Plugin event...
Plugin event...
Plugin event...
Plugin event...
Plugin event...
Plugin event...
Plugin event...
Station CA-304 is occupied with ev 2_39_138_29_2018-09-01 15:31:50.661778.
Plugin event...
Station CA-304 is occupied with ev 2_39_138_29_2018-09-01 15:31:50.661778.
Plugin event...
Station CA-303 is occupied with ev 2_39_139_28_2018-09-01 15:28:41.249325.
Plugin event...
Station CA-494 is occupied with ev 2_39_78_367_2018-09-01 10:49:04.972619.
Plugin event...
Plugin event...
Station CA-306 is occupied with ev 2_39_130_31_2018-09-01 14:33:40.470366.
Plugined in EV 15


In [10]:
# Debugging/Observing some variables
print(network.active_evs)
# print(network.current_charging_rates)
# print(network.constraints_as_df())

[acnportal.acnsim.models.ev.EV(_arrival=101, _departure=110, _session_id=<str object at 0x000002677D22F7B0>, _station_id=<str object at 0x000002677D234070>, _requested_energy=4.858, _estimated_departure=110, _battery=<acnportal.acnsim.models.battery.Battery object at 0x000002677DA8E7C0>, _energy_delivered=0, _current_charging_rate=0), acnportal.acnsim.models.ev.EV(_arrival=90, _departure=183, _session_id=<str object at 0x000002677D22F690>, _station_id=<str object at 0x000002677D232CF0>, _requested_energy=3.511, _estimated_departure=183, _battery=<acnportal.acnsim.models.battery.Battery object at 0x000002677DA8E4C0>, _energy_delivered=0, _current_charging_rate=0), acnportal.acnsim.models.ev.EV(_arrival=1, _departure=116, _session_id=<str object at 0x000002677D212810>, _station_id=<str object at 0x000002677D22E6F0>, _requested_energy=37.14, _estimated_departure=116, _battery=<acnportal.acnsim.models.battery.Battery object at 0x000002677D919610>, _energy_delivered=0, _current_charging_rat

### Simplifying variables for optimization algorithm

In [11]:
# Final simulation setup
basic_evse = True
estimate_max_rate = False
uninterrupted_charging = False
quantized = False
constraint_type = 'SOC'
enforce_energy_equality=False
solver='MOSEK',
peak_limit = None
prev_peak = 0
tariff_name = 'sce_tou_ev_4_march_2019'

In [12]:
def get_active_sessions(active_evs, current_time):
    """Return a list of SessionInfo objects describing the currently charging EVs.

    Args:
        active_evs (List[acnsim.EV]: List of EV objects from acnsim.
        current_time (int): Current time of the simulation.

    Returns:
        List[SessionInfo]: List of currently active charging sessions.
    """
    return [
        SessionInfo(
            ev.station_id,
            ev.session_id,
            ev.requested_energy,
            ev.energy_delivered,
            ev.arrival,
            ev.departure,
            current_time=current_time,
        )
        for ev in active_evs
    ]

In [42]:
# sessions = get_active_sessions(network.active_evs, 0)
# print(sessions)
# print(len(sessions))

[<acnportal.acnsim.interface.SessionInfo object at 0x000001D27EC036D0>, <acnportal.acnsim.interface.SessionInfo object at 0x000001D27EC03D90>, <acnportal.acnsim.interface.SessionInfo object at 0x000001D27EC03190>, <acnportal.acnsim.interface.SessionInfo object at 0x000001D27EC03DF0>, <acnportal.acnsim.interface.SessionInfo object at 0x000001D27EC03730>, <acnportal.acnsim.interface.SessionInfo object at 0x000001D27EC03E50>, <acnportal.acnsim.interface.SessionInfo object at 0x000001D27EC03B50>, <acnportal.acnsim.interface.SessionInfo object at 0x000001D27EC037F0>, <acnportal.acnsim.interface.SessionInfo object at 0x000001D27EC03910>, <acnportal.acnsim.interface.SessionInfo object at 0x000001D27EC030D0>, <acnportal.acnsim.interface.SessionInfo object at 0x000001D27EC03940>, <acnportal.acnsim.interface.SessionInfo object at 0x000001D27EC03C40>, <acnportal.acnsim.interface.SessionInfo object at 0x000001D27EC03F40>, <acnportal.acnsim.interface.SessionInfo object at 0x000001D27EC03BB0>, <acnp

In [13]:
# System Information object for algorithm

station_ids = network.station_ids
max_pilot_signals = network.max_pilot_signals
min_pilot_signals = network.min_pilot_signals
allowable_rates = network.allowable_rates
is_continuous = network.is_continuous

infrastructure = InfrastructureInfo(
                    network.constraint_matrix,
                    network.magnitudes,
                    network._phase_angles,
                    network._voltages,
                    network.constraint_index,
                    station_ids,
                    max_pilot_signals,
                    min_pilot_signals,
                    allowable_rates,
                    is_continuous,
                )
print(infrastructure.voltages)

[208. 208. 208. 208. 208. 208. 208. 208. 208. 208. 208. 208. 208. 208.
 208. 208. 208. 208. 208. 208. 208. 208. 208. 208. 208. 208. 208. 208.
 208. 208. 208. 208. 208. 208. 208. 208. 208. 208. 208. 208. 208. 208.
 208. 208. 208. 208. 208. 208. 208. 208. 208. 208. 208. 208.]


In [44]:
# Preprocessing for the algorithm   
def enforce_pilot_limit(
    active_sessions: List[SessionInfo], infrastructure: InfrastructureInfo
) -> List[SessionInfo]:
    """ Update the max_rates vector for each session to be less than the max
        pilot supported by its EVSE.

    Args:
        active_sessions (List[SessionInfo]): List of SessionInfo objects for
            all active charging sessions.
        infrastructure (InfrastructureInfo): Description of the charging
            infrastructure.

    Returns:
        List[SessionInfo]: Active sessions with max_rates updated to be at
            most the max_pilot of the corresponding EVSE.
    """
    for session in active_sessions:
        i = infrastructure.get_station_index(session.station_id)
        session.max_rates = np.minimum(session.max_rates, infrastructure.max_pilot[i])
    return active_sessions

In [45]:
# Enforcing pilot limit for sessions
# active_sessions = enforce_pilot_limit(sessions, infrastructure)

In [15]:
# For offline solution
active_evs = [
            deepcopy(event.ev)
            for event in processed_event
            if event.event_type == "Plugin"
        ]
sessions = get_active_sessions(active_evs, 0)
session_ids = set(s.session_id for s in sessions)

In [16]:
print(sessions)
print(session_ids)

[<acnportal.acnsim.interface.SessionInfo object at 0x000002677DE404C0>, <acnportal.acnsim.interface.SessionInfo object at 0x000002677DE40700>, <acnportal.acnsim.interface.SessionInfo object at 0x000002677DE40340>, <acnportal.acnsim.interface.SessionInfo object at 0x000002677D8D4400>, <acnportal.acnsim.interface.SessionInfo object at 0x000002677D8D4EE0>, <acnportal.acnsim.interface.SessionInfo object at 0x000002677D8D46A0>, <acnportal.acnsim.interface.SessionInfo object at 0x000002677D8D4250>, <acnportal.acnsim.interface.SessionInfo object at 0x000002677D8D4130>, <acnportal.acnsim.interface.SessionInfo object at 0x000002677D8D41C0>, <acnportal.acnsim.interface.SessionInfo object at 0x000002677D8D4430>, <acnportal.acnsim.interface.SessionInfo object at 0x000002677D8D41F0>, <acnportal.acnsim.interface.SessionInfo object at 0x000002677D8D4B20>, <acnportal.acnsim.interface.SessionInfo object at 0x000002677D8D4B50>, <acnportal.acnsim.interface.SessionInfo object at 0x000002677D8D4CA0>, <acnp

### Objective functions

In [17]:
objectives = [
    ObjectiveComponent(quick_charge),
    ObjectiveComponent(equal_share, 1e-12)
]

### Necessary utility functions

In [22]:
# Charging rate bounds
def charging_rate_bounds(
        rates: cp.Variable, active_sessions: List[SessionInfo], evse_index: List[str]
    ):
        """Get upper and lower bound constraints for each charging rate.

        Args:
            rates (cp.Variable): cvxpy variable representing all charging rates. Shape should be (N, T) where N is the
                total number of EVSEs in the system and T is the length of the optimization horizon.
            active_sessions (List[SessionInfo]): List of SessionInfo objects for all active charging sessions.
            evse_index (List[str]): List of IDs for all EVSEs. Index in evse_index represents the row number of that
                EVSE in rates.

        Returns:
            List[cp.Constraint]: List of lower bound constraint, upper bound constraint.
        """
        lb, ub = np.zeros(rates.shape), np.zeros(rates.shape)
        for session in active_sessions:
            i = evse_index.index(session.station_id)
            lb[
                i,
                session.arrival_offset : session.arrival_offset
                + session.remaining_time,
            ] = session.min_rates
            ub[
                i,
                session.arrival_offset : session.arrival_offset
                + session.remaining_time,
            ] = session.max_rates
        # To ensure feasibility, replace upper bound with lower bound when they conflict
        ub[ub < lb] = lb[ub < lb]
        return {
            "charging_rate_bounds.lb": rates >= lb,
            "charging_rate_bounds.ub": rates <= ub,
        }


# Energy constraints
def energy_constraints(
        rates: cp.Variable,
        active_sessions: List[SessionInfo],
        infrastructure: InfrastructureInfo,
        period,
        enforce_energy_equality=False,
    ):
        """Get constraints on the energy delivered for each session.

        Args:
            rates (cp.Variable): cvxpy variable representing all charging rates. Shape should be (N, T) where N is the
                total number of EVSEs in the system and T is the length of the optimization horizon.
            active_sessions (List[SessionInfo]): List of SessionInfo objects for all active charging sessions.
            infrastructure (InfrastructureInfo): InfrastructureInfo object describing the electrical infrastructure at
                a site.
            period (int): Length of each discrete time period. (min)
            enforce_energy_equality (bool): If True, energy delivered must be equal to energy requested for each EV.
                If False, energy delivered must be less than or equal to request.

        Returns:
            List[cp.Constraint]: List of energy delivered constraints for each session.
        """
        constraints = {}
        for session in active_sessions:
            i = infrastructure.get_station_index(session.station_id)
            planned_energy = cp.sum(
                rates[
                    i,
                    session.arrival_offset : session.arrival_offset
                    + session.remaining_time,
                ]
            )
            planned_energy *= infrastructure.voltages[i] * period / 1e3 / 60
            constraint_name = f"energy_constraints.{session.session_id}"
            if enforce_energy_equality:
                constraints[constraint_name] = (
                    planned_energy == session.remaining_demand
                )
            else:
                constraints[constraint_name] = (
                    planned_energy <= session.remaining_demand
                )
        return constraints


# Infrastructure constraints
def infrastructure_constraints(
        rates: cp.Variable, infrastructure: InfrastructureInfo, constraint_type="SOC"
    ):
        """Get constraints enforcing infrastructure limits.

        Args:
            rates (cp.Variable): cvxpy variable representing all charging rates. Shape should be (N, T) where N is the
                total number of EVSEs in the system and T is the length of the optimization horizon.
            infrastructure (InfrastructureInfo): InfrastructureInfo object describing the electrical infrastructure at
                a site.
            constraint_type (str): String representing which constraint type to use. Options are 'SOC' for Second Order
                Cone or 'LINEAR' for linearized constraints.

        Returns:
            List[cp.Constraint]: List of constraints, one for each bottleneck in the electrical infrastructure.
        """
        # If constraint_matrix is empty, no need to add infrastructure
        # constraints.
        if (
            infrastructure.constraint_matrix is None
            or infrastructure.constraint_matrix.shape == (0, 0)
        ):
            return {}
        constraints = {}
        if constraint_type == "SOC":
            if infrastructure.phases is None:
                raise ValueError(
                    "phases is required when using SOC infrastructure constraints."
                )
            phase_in_rad = np.deg2rad(infrastructure.phases)
            for j, v in enumerate(infrastructure.constraint_matrix):
                a = np.stack([v * np.cos(phase_in_rad), v * np.sin(phase_in_rad)])
                constraint_name = (
                    f"infrastructure_constraints." f"{infrastructure.constraint_ids[j]}"
                )
                constraints[constraint_name] = (
                    cp.norm(a @ rates, axis=0) <= infrastructure.constraint_limits[j]
                )
        elif constraint_type == "LINEAR":
            for j, v in enumerate(infrastructure.constraint_matrix):
                constraint_name = (
                    f"infrastructure_constraints.{infrastructure.constraint_ids[j]}"
                )
                constraints[constraint_name] = (
                    np.abs(v) @ rates <= infrastructure.constraint_limits[j]
                )
        else:
            raise ValueError(
                "Invalid infrastructure constraint type: {0}. Valid options are SOC or AFFINE.".format(
                    constraint_type
                )
            )
        return constraints

# Peak constratint 
def peak_constraint(
        rates: cp.Variable, peak_limit: Union[float, List[float], np.ndarray]
    ):
        """Get constraints enforcing infrastructure limits.

        Args:
            rates (cp.Variable): cvxpy variable representing all charging rates. Shape should be (N, T) where N is the
                total number of EVSEs in the system and T is the length of the optimization horizon.
            peak_limit (Union[float, List[float], np.ndarray]): Limit on aggregate peak current. If None, no limit is
                enforced.

        Returns:
            List[cp.Constraint]: List of constraints, one for each bottleneck in the electrical infrastructure.
        """
        if peak_limit is not None:
            return {"peak_constraint": cp.sum(rates, axis=0) <= peak_limit}
        return {}


def project_into_continuous_feasible_pilots(
    rates: np.ndarray, infrastructure: InfrastructureInfo
):
    """Round all values in rates such that they are less than the max_pilot of the corresponding EVSE and greater than
            or equal to 0.

    Args:
        rates (np.ndarray): Schedule of charging rates.
        infrastructure (InfrastructureInfo): Description of the charging infrastructure.

    Returns:
        np.ndarray: Rounded schedule of charging rates.
    """
    new_rates = deepcopy(rates)
    for i in range(infrastructure.num_stations):
        new_rates[i] = np.minimum(rates[i], infrastructure.max_pilot[i])
    new_rates = np.maximum(new_rates, 0)
    return new_rates

### Solving Optimization Problem

In [23]:
interface = Interface

def build_objective(
        rates: cp.Variable, infrastructure: InfrastructureInfo, **kwargs
    ):
        def _merge_dicts(*args):
            """ Merge two dictionaries where d2 override d1 when there is a conflict. """
            merged = dict()
            for d in args:
                merged.update(d)
            return merged

        obj = cp.Constant(0)
        for component in objectives:
            obj += component.coefficient * component.function(
                rates,
                infrastructure,
                interface,
                **_merge_dicts(kwargs, component.kwargs),
            )
        return obj

In [24]:
def build_problem(
        active_sessions: List[SessionInfo],
        infrastructure: InfrastructureInfo,
        peak_limit: Optional[Union[float, List[float], np.ndarray]] = None,
        prev_peak: float = 0,
    ):
        """Build parts of the optimization problem including variables, constraints, and objective function.

        Args:
            active_sessions (List[SessionInfo]): List of SessionInfo objects for all active charging sessions.
            infrastructure (InfrastructureInfo): InfrastructureInfo object describing the electrical infrastructure at
                a site.
            peak_limit (Union[float, List[float], np.ndarray]): Limit on aggregate peak current. If None, no limit is
                enforced.
            prev_peak (float): Previous peak current draw during the current billing period.

        Returns:
            Dict[str: object]:
                'objective' : cvxpy expression for the objective of the optimization problem
                'constraints': list of all constraints for the optimization problem
                'variables': dict mapping variable name to cvxpy Variable.
        """
        optimization_horizon = max(
            s.arrival_offset + s.remaining_time for s in active_sessions
        )
        num_evses = len(infrastructure.station_ids)
        rates = cp.Variable(shape=(num_evses, optimization_horizon))
        constraints = {}

        # Rate constraints
        constraints.update(
            charging_rate_bounds(
                rates, active_sessions, infrastructure.station_ids
            )
        )

        # Energy Delivered Constraints
        constraints.update(
            energy_constraints(
                rates,
                active_sessions,
                infrastructure,
                PERIOD,
                enforce_energy_equality,
            )
        )

        # Infrastructure Constraints
        constraints.update(
            infrastructure_constraints(rates, infrastructure, constraint_type)
        )

        # Peak Limit
        constraints.update(peak_constraint(rates, peak_limit))

        # Objective Function
        objective = cp.Maximize(
            build_objective(rates, infrastructure, prev_peak=prev_peak)
        )
        return {
            "objective": objective,
            "constraints": constraints,
            "variables": {"rates": rates},
        }

In [ ]:
# problem_dict = build_problem(active_sessions, infrastructure, peak_limit)
# print(problem_dict)

In [25]:
problem_dict = build_problem(sessions, infrastructure, peak_limit)
print(problem_dict)

{'objective': Maximize(Expression(CONCAVE, UNKNOWN, ())), 'constraints': {'charging_rate_bounds.lb': Inequality(Constant(CONSTANT, ZERO, (54, 216))), 'charging_rate_bounds.ub': Inequality(Variable((54, 216))), 'energy_constraints.2_39_79_381_2018-09-01 07:05:59.297327': Inequality(Expression(AFFINE, UNKNOWN, ())), 'energy_constraints.2_39_131_30_2018-09-01 07:10:56.837854': Inequality(Expression(AFFINE, UNKNOWN, ())), 'energy_constraints.2_39_78_367_2018-09-01 10:49:04.972619': Inequality(Expression(AFFINE, UNKNOWN, ())), 'energy_constraints.2_39_78_365_2018-09-01 13:36:45.223594': Inequality(Expression(AFFINE, UNKNOWN, ())), 'energy_constraints.2_39_89_25_2018-09-01 13:47:23.273995': Inequality(Expression(AFFINE, UNKNOWN, ())), 'energy_constraints.2_39_95_27_2018-09-01 14:27:19.797274': Inequality(Expression(AFFINE, UNKNOWN, ())), 'energy_constraints.2_39_130_31_2018-09-01 14:33:40.470366': Inequality(Expression(AFFINE, UNKNOWN, ())), 'energy_constraints.2_39_79_379_2018-09-01 14:34:2

In [ ]:
# Offline 
prob = cp.Problem(problem_dict["objective"], list(problem_dict["constraints"].values()))
rates_matrix = prob.solve(solver=cp.ECOS_BB, verbose=True)
print(rates_matrix)

In [ ]:
# prob = cp.Problem(
#             problem_dict["objective"], list(problem_dict["constraints"].values())
#         )
# rates_matrix = prob.solve(solver=cp.MOSEK, verbose=VERBOSE)
# # rates_matrix = project_into_continuous_feasible_pilots(rates_matrix, infrastructure)
# # internal_schedule = {
# #             station_id: rates_matrix[i, :]
# #             for i, station_id in enumerate(infrastructure.station_ids)
# #         }
# print(rates_matrix)

In [ ]:
# Alternate method
# prob = cp.Problem(
#             problem_dict["objective"], list(problem_dict["constraints"].values())
#         )
# prob.solve(solver=cp.MOSEK, verbose=VERBOSE)
# if prob.status not in [cp.OPTIMAL, cp.OPTIMAL_INACCURATE]:
#             raise InfeasibilityException(f"Solve failed with status {prob.status}")
# else:
#     print(problem_dict["variables"]["rates"].value)